In [ ]:
#!pip install tensorflow==2.17.0

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf # 2.17.0
print(tf.__version__)
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D, Add, concatenate, Lambda
from tensorflow.keras.models import Model
import importlib.util
import sys

# Define the path to your script
utils_path = './utils.py'

# Load the module
spec = importlib.util.spec_from_file_location("utils", utils_path)
utils = importlib.util.module_from_spec(spec)
sys.modules["utils"] = utils
spec.loader.exec_module(utils)
from utils import process_outputs, draw_boxes, make_yolov3_model, _conv_block, yolo_head
# Load the NumPy array from the file
anchors = np.load('./anchors.npy')

In [ ]:
model = make_yolov3_model()
# Input tensor
inputs = model.input

x = model.get_layer('leaky_80').output
yolo_82 = _conv_block(x, [{'filter':  20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

x = model.get_layer('leaky_92').output
yolo_94 = _conv_block(x, [{'filter': 20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

x = model.get_layer('leaky_104').output
yolo_106 = _conv_block(x, [{'filter': 20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)

# Create a new model with the modified architecture
model = Model(inputs=inputs, outputs=[yolo_82, yolo_94, yolo_106])
model.load_weights('./model.weights.h5')

In [ ]:
test_img = cv2.imread('your/input/image/path')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

In [ ]:
img = cv2.resize(test_img, (608,608))
pred = model.predict(img.reshape(1,608,608,3))
boxes, confidences = process_outputs(pred, [anchors,anchors,anchors], confidence_thresh=0.5, iou_thresh=0.00)
im_out =  draw_boxes(test_img.copy(), boxes, confidences)

In [ ]:
plt.imshow(im_out)